In [1]:
if True:
    %rm -rf source
    !git clone https://github.com/HaiDang2001VN/albert-imdb.git source
    %pip install -U wandb transformers evaluate huggingface_hub accelerate
    colab = True
else:
    print("Not running on Google Colab, skip this cell!")
    colab = False

Cloning into 'source'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 193 (delta 100), reused 158 (delta 69), pack-reused 0
Receiving objects: 100% (193/193), 1.76 MiB | 4.65 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [2]:
import json
from pprint import pprint

# Load path to pre-processed data in configs folder
config_file = "vlsp_colab" if colab else "vlsp_local"
with open(f"configs/{config_file}.json") as f:
    configs = json.load(f)

pprint(configs)

{'model_path': {'albert': {'base': 'albert-base-v2',
                           'large': 'albert-large-v2'},
                'bart': {'base': 'facebook/bart-base',
                         'large': 'facebook/bart-large'},
                'electra': {'base': 'google/electra-base-discriminator',
                            'large': 'google/electra-large-discriminator',
                            'small': 'google/electra-small-discriminator'},
                'phobert': {'base': 'vinai/phobert-base',
                            'large': 'vinai/phobert-large'}},
 'path': '/content/source'}


In [3]:
import wandb

run_model = ["albert", "base"]
wandb.init(project="advanced_ai_imdb_dataset", name=f"{run_model[0]}-{run_model[1]}-run-vlsp")

wandb: Currently logged in as: joatmon. Use `wandb login --relogin` to force relogin


In [4]:
from datasets import load_dataset

source_path = configs["path"]
format_name = "format0"
# format_name = "format1"
# train_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/train.jsonl", split="train"
# )
# val_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/dev.jsonl", split="train"
# )
# test_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/test.jsonl", split="train"
# )
dataset = load_dataset(
    "json",
    data_files={
        "train": f"{source_path}/vlsp_preprocessed/{format_name}/train.jsonl",
        "val": f"{source_path}/vlsp_preprocessed/{format_name}/dev.jsonl",
        "test": f"{source_path}/vlsp_preprocessed/{format_name}/test.jsonl",
    },
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
model_path = configs['model_path'][run_model[0]][run_model[1]]
print(model_path)

albert-base-v2


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=4,
                                                           output_hidden_states=False
                                                           )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def preprocess_token(example):
    return tokenizer(
        example["sentence"], padding="max_length", truncation=True, return_tensors="pt"
    )

In [ ]:
tokenized_dataset = dataset.map(preprocess_token, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(f"Type of {type(predictions)=}")
    print(f"Type of {type(labels)=}")
    print(f"Shape of {predictions[0].shape}")
    print(f"Shape of {predictions[1].shape}")
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    predictions = np.argmax(predictions, axis=1)
    # predictions = np.argmax(predictions.reshape(-1, predictions.shape[-1]), axis=1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)['accuracy'],
        "f1_macro": f1.compute(predictions=predictions, references=labels, average="macro")['f1'],
        "f1_micro": f1.compute(predictions=predictions, references=labels, average="micro")['f1'],
        "f1_weighted": f1.compute(predictions=predictions, references=labels, average="weighted")['f1'],
        "precision_macro": precision.compute(predictions=predictions, references=labels, average="macro")['precision'],
        "precision_micro": precision.compute(predictions=predictions, references=labels, average="micro")['precision'],
        "precision_weighted": precision.compute(predictions=predictions, references=labels, average="weighted")['precision'],
        "recall_macro": recall.compute(predictions=predictions, references=labels, average="macro")['recall'],
        "recall_micro": recall.compute(predictions=predictions, references=labels, average="micro")['recall'],
        "recall_weighted": recall.compute(predictions=predictions, references=labels, average="weighted")['recall'],
    }

In [ ]:
id2label = {0: 'AFFILIATION', 1: 'PART – WHOLE', 2: 'LOCATED', 3: 'PERSONAL - SOCIAL'}
label2id = {value: key for key, value in id2label.items()}
print(label2id)

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="albert_imdb",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    eval_steps=500,
    save_total_limit=1,
    eval_accumulation_steps=10,
    report_to="wandb",
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [ ]:
tokenized_dataset["train"][0]

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.evaluate(tokenized_dataset["test"], metric_key_prefix="test")

In [ ]:
wandb.finish()